In [33]:
from gensim import corpora, models, similarities, matutils
import itertools
import numpy as np
import pandas as pd

In [2]:
dropbox = "/Users/mr/Dropbox/moviemeta/"

In [14]:
lda = models.LdaModel.load(dropbox +'lda_imdb_2015.model')
corpus = corpora.MmCorpus(dropbox +'lda_imdb_2015.corpus')
dictionary = corpora.Dictionary.load(dropbox +'lda_imdb_2015.dict')
with open(dropbox +'imdb_movies_since_2014.pickle', 'rb') as f:
    movies = np.load(f)

In [74]:
topic_matrix = matutils.corpus2dense(lda[corpus], num_terms=20)
topic_df = pd.DataFrame(np.ndarray.transpose(topic_matrix))
topic_df['movie_title'] = pd.Series(movies, index=topic_df.index)

Now we have created a dataframe with the topic distribution for every movie. We have 20 topics, columns 0 - 19. The values represent how prominently a topic features in a movie.

In [84]:
topic_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,movie_title
0,0,0,0.482269,0.000,0.000000,0.000000,0.000000,0.177444,0.000000,0.000000,...,0.000000,0,0.000000,0,0.000000,0,0.000000,0.000000,0.283620,#1 at the Apocalypse Box Office (2015)
1,0,0,0.000000,0.039,0.000000,0.000000,0.000000,0.000000,0.078759,0.569357,...,0.173276,0,0.000000,0,0.041973,0,0.000000,0.084173,0.000000,#50Fathers (2015)
2,0,0,0.000000,0.000,0.077895,0.000000,0.228392,0.000000,0.214340,0.000000,...,0.000000,0,0.000000,0,0.000000,0,0.459374,0.000000,0.000000,#BeRobin the Movie (2015)
3,0,0,0.000000,0.000,0.000000,0.092356,0.000000,0.267722,0.330263,0.000000,...,0.000000,0,0.298392,0,0.000000,0,0.000000,0.000000,0.000000,#Beings (2015)
4,0,0,0.000000,0.000,0.214896,0.000000,0.000000,0.000000,0.000000,0.538686,...,0.000000,0,0.000000,0,0.000000,0,0.000000,0.000000,0.211001,#Disneyland60 (2015)


In [85]:
lda.print_topics(20)

[u'0.009*zombi + 0.009*japanes + 0.008*camp + 0.007*surviv + 0.007*human + 0.006*medic + 0.004*patient + 0.004*diseas + 0.004*scientist + 0.004*death',
 u'0.006*novel + 0.005*club + 0.005*master + 0.005*plan + 0.004*villag + 0.004*night + 0.004*play + 0.004*love + 0.004*write + 0.004*chang',
 u'0.015*war + 0.010*human + 0.007*earth + 0.006*surviv + 0.006*race + 0.006*journey + 0.006*american + 0.006*forc + 0.005*fight + 0.005*anim',
 u'0.012*friend + 0.009*play + 0.009*comedi + 0.007*love + 0.005*start + 0.005*relationship + 0.005*girl + 0.005*parti + 0.004*charact + 0.004*success',
 u'0.011*documentari + 0.011*citi + 0.008*histori + 0.007*rock + 0.006*industri + 0.006*activist + 0.005*right + 0.005*band + 0.005*punk + 0.005*includ',
 u'0.039*love + 0.014*relationship + 0.010*woman + 0.010*marri + 0.009*famili + 0.008*coupl + 0.007*fall + 0.007*marriag + 0.007*decid + 0.006*dream',
 u'0.030*music + 0.018*artist + 0.015*art + 0.009*perform + 0.007*record + 0.006*musician + 0.006*creat +